In [3]:
import nltk
import pandas as pd
import numpy as np
import sys

ids = pd.read_csv('car_model_identifier.csv', header=0)
ids = ids.sort_values(by=['model_id'])
data = pd.read_csv('entire_data_Mentioned_4th.csv', header=0).iloc[:, 1:] ## because the first column are indexes

import re
from mlxtend.preprocessing import TransactionEncoder
te = TransactionEncoder()

In [84]:
ids

,brand,model_fromsales,year,model_id,brand_id,yb_id,ybm_id
0,ACURA,CL,2002,1,1,1,1
1,ACURA,CL,2003,1,1,2,194
2,ACURA,ILX,2013,2,1,12,2219
3,ACURA,MDX,2002,3,1,1,2
4,ACURA,MDX,2003,3,1,2,195
...,...,...,...,...,...,...,...
2382,VOLVO,XC90,2005,416,40,393,798
2381,VOLVO,XC90,2004,416,40,392,589
2380,VOLVO,XC90,2003,416,40,391,378
2384,VOLVO,XC90,2007,416,40,395,1221


## Generate a list of model names

In [85]:
unique_model=[]
for x in ids.model_fromsales:
    if x not in unique_model:
         unique_model.append(x)
len(unique_model)

416

In [105]:
print(unique_model)

['CL', 'ILX', 'MDX', 'NSX', 'RDX', 'RL', 'RSX', 'TL', 'TSX', 'ZDX', 'A3', 'A4', 'A6', 'TT', '1 SERIES', '3 SERIES', '5 SERIES', '6 SERIES', '7 SERIES', 'X1', 'X3', 'X5', 'X6', 'Z3', 'Z4', 'Z8', 'CENTURY', 'ENCLAVE', 'ENCORE', 'LACROSSE', 'LESABRE', 'LUCERNE', 'PARK AVE', 'RAINIER', 'REGAL', 'RENDEZVOUS', 'TERRAZA', 'VERANO', 'ATS', 'CTS', 'DEVILLE', 'DTS', 'ELDORADO', 'ESCALADE', 'ESCALADE ESV', 'ESCALADE EXT', 'SEVILLE', 'SRX', 'STS', 'XLR', 'XTS', 'ASTRO PASS', 'AVALANCHE', 'AVEO', 'CAMARO', 'CAVALIER', 'COBALT', 'COLORADO', 'CORVETTE', 'CRUZE', 'EQUINOX', 'EXPRESS CARGO', 'EXPRESS PASS', 'HHR', 'IMPALA', 'MALIBU', 'MONTE CARLO', 'PRIZM', 'S BLAZER', 'S10 PICKUP', 'SILVERADO', 'SONIC', 'SPARK', 'SSR', 'SUBURBAN', 'TAHOE', 'TRACKER', 'TRAILBLAZER', 'TRAVERSE', 'UPLANDER', 'VENTURE', 'VOLT', '200 SERIES', '300 M', '300 SERIES', 'ASPEN', 'CONCORDE', 'CROSSFIRE', 'PACIFICA', 'PT CRUISER', 'SEBRING SEDAN', 'TOWN & COUNTRY', 'VOYAGER', 'LANOS', 'LEGANZA', 'NUBIRA', 'AVENGER', 'CALIBER', 'C

## add 40 new columns to the dataframe, each named by a brand name

In [125]:
data = pd.concat([data, pd.DataFrame(columns=unique_model)])
data = data0.replace(np.nan, 0)
data.model_id = data.model_id.astype(int)

In [120]:
len(data.columns) ## 7 descriptive columns plus 416 brand columns

423

### check if the detected brand_id in the review equals to the brand_id of a brand name

In [127]:
import time

t1 = time.time()

for i in range(len(data)):
    for y in range (416):
        if data.model_id[i] == y+1:
             data.iloc[i,y+7] = 1 ## because the model columns starts from the 7th column 
            
t2 = time.time()
print(t2 - t1)

855.2531158924103


In [128]:
data

,detected_entity,year,model_id,brand_id,yb_id,ybm_id,org_id,CL,ILX,MDX,...,30,40,50,60,70 C,70 V,80,XC60,XC70,XC90
0,CL,2002.0,1,1.0,1.0,1.0,185.0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ACURA,2002.0,1,1.0,1.0,1.0,150328.0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,MDX,2002.0,3,1.0,1.0,2.0,1851.0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,MDX,2002.0,3,1.0,1.0,2.0,1858.0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,MDX,2002.0,3,1.0,1.0,2.0,2109.0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105916,MDX,0.0,3,1.0,0.0,0.0,310085.0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
105917,VOLVO,0.0,412,40.0,0.0,0.0,310087.0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
105918,VOLVO,0.0,412,40.0,0.0,0.0,310091.0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
105919,BMW,0.0,22,3.0,0.0,0.0,310093.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [129]:
data.to_csv('PyShark_model2.csv')


## get only the brand columns and org_ids

In [135]:
data = pd.read_csv('PyShark_model2.csv', header=0).iloc[:, 1:] ## because the first column are indexes
data_record = data.iloc[0:,6:]

In [137]:
data.head()

,detected_entity,year,model_id,brand_id,yb_id,ybm_id,org_id,CL,ILX,MDX,...,30,40,50,60,70 C,70 V,80,XC60,XC70,XC90
0,CL,2002.0,1,1.0,1.0,1.0,185.0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ACURA,2002.0,1,1.0,1.0,1.0,150328.0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,MDX,2002.0,3,1.0,1.0,2.0,1851.0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,MDX,2002.0,3,1.0,1.0,2.0,1858.0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,MDX,2002.0,3,1.0,1.0,2.0,2109.0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [136]:
data_record.head()

,org_id,CL,ILX,MDX,NSX,RDX,RL,RSX,TL,TSX,...,30,40,50,60,70 C,70 V,80,XC60,XC70,XC90
0,185.0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,150328.0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1851.0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1858.0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2109.0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## add up the occurences of brand names in a single review (group by org_id)

In [138]:
data_condensed=data_record.groupby(['org_id']).sum()
data_condensed.to_csv("PyShark_condensed_model2.csv")

## replace all values > 1 with 1

In [141]:
data_condensed = data_condensed.apply(lambda x: [y if y <= 1 else 1 for y in x])  ## replace all values > 1 with 1 

In [35]:
from itertools import combinations

class lift_calculator:
    def __init__(self, data):
        self.data = data
        self.brand_count = dict(data.sum(axis=0))
        self.pair_brand_count = dict()
        for brand1, brand2 in combinations(data.columns, 2):
            self.pair_brand_count[(brand1, brand2)] = (data[brand1] * data[brand2]).sum()
        self.lift_dict = dict()
        self.total_size = len(data)
        
    def calculate(self):
        for brand1, brand2 in self.pair_brand_count:
            self.lift_dict[(brand1, brand2)] = self.pair_brand_count[(brand1, brand2)] * self.total_size / (self.brand_count[brand1] * self.brand_count[brand2])
    
    def top_n(self, n=10):
        data = pd.DataFrame(self.lift_dict.items()).fillna(0).sort_values(by=1, ascending=False).iloc[:n, :]
        top_n_pairs = dict(zip(data[0], data[1]))
        return top_n_pairs
    
    def return_csv(self):
        df = pd.DataFrame(index=data.columns, columns=data.columns)
        for brand1 in data.columns:
            for brand2 in data.columns:
                df.loc[brand1, brand2] = self.lift_dict.get((brand1, brand2), 0)
        return df

In [37]:
data_condensed

,CL,ILX,MDX,NSX,RDX,RL,RSX,TL,TSX,ZDX,...,30,40,50,60,70 C,70 V,80,XC60,XC70,XC90
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63802,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
63803,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
63804,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
63805,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [38]:
data = data_condensed
# pd.read_csv("PyShark_condensed_model.csv", index_col=0)
calculator = lift_calculator(data)
calculator.calculate()

/Users/shijia_huang/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in long_scalars
  from ipykernel import kernelapp as app


In [18]:
lift_model_df = calculator.return_csv()

lift_model_df.to_csv("PyShark_paired_model_lift_no_diagonal2.csv")

In [1]:
lift_model_df

NameError: name 'lift_model_df' is not defined

In [42]:
calculator.top_n(50)

{('STRATUS COUPE', 'STRATUS SEDAN'): 2058.2903225806454,
 ('DART', 'BRZ'): 1519.2142857142858,
 ('LX470', 'CLK CLASS'): 886.2083333333334,
 ('MONTE CARLO', 'RODEO'): 862.2567567567568,
 ('G CLASS', '9-2X'): 818.0384615384615,
 ('ZDX', 'RX400'): 797.5875,
 ('350Z', 'MR2 SPYDER'): 797.5875,
 ('Q45', 'HS250'): 787.7407407407408,
 ('SONIC', 'MAZDA2'): 733.4137931034483,
 ('ZDX', 'X6'): 708.9666666666667,
 ('C CLASS', 'R CLASS'): 664.65625,
 ('FIESTA', 'MAZDA2'): 518.7560975609756,
 ('X6', 'RX400'): 443.1041666666667,
 ('CHALLENGER', 'BRZ'): 409.0192307692308,
 ('ESCORT', 'ECHO'): 335.8263157894737,
 ('R CLASS', '30'): 332.328125,
 ('SEVILLE', 'XK8'): 327.2153846153846,
 ('VOLT', 'CT200'): 327.2153846153846,
 ('WINDSTAR PASS', 'SAVANA PASS'): 315.8762376237624,
 ('I35', 'ES300'): 314.320197044335,
 ('SRX', 'LX570'): 305.29665071770336,
 ('PASSPORT', 'RODEO'): 287.4189189189189,
 ('COUNTRYMAN', 'JUKE'): 263.6652892561983,
 ('XK8', 'SL CLASS'): 256.94093959731543,
 ('LEGANZA', 'VERONA'): 255.

## we have about 16842 pairs of models that have a nonzero lift

In [148]:
 np.count_nonzero(lift_model_df)

16842

### follow the instruction from Pyshark:  (basically the ap and fp methed generates the same results except ordered differently)
### https://pyshark.com/market-basket-analysis-using-association-rule-mining-in-python/

In [151]:
from mlxtend.frequent_patterns import apriori
frequent_itemsets_ap = apriori(data_condensed,min_support=0.001, use_colnames=True) ## the min_support could be modified

In [152]:
frequent_itemsets_ap

,support,itemsets
0,0.002805,(CL)
1,0.013400,(MDX)
2,0.001583,(RDX)
3,0.003746,(RL)
4,0.004294,(RSX)
...,...,...
276,0.001395,"(FR-S, PRIUS C)"
277,0.001912,"(CAMRY, PRIUS C)"
278,0.001019,"(PASSAT, 70 V)"
279,0.001379,"(ACCORD, CAMRY, ALTIMA)"


In [153]:
from mlxtend.frequent_patterns import fpgrowth
frequent_itemsets_fp=fpgrowth(data_condensed, min_support=0.001, use_colnames=True) ## the min_support could be modified

In [154]:
print(frequent_itemsets_fp)

      support                 itemsets
0    0.013306               (5 SERIES)
1    0.002805                     (CL)
2    0.006723                 (G35/37)
3    0.002335               (SL CLASS)
4    0.011801                (MUSTANG)
..        ...                      ...
276  0.001442        (C-MAX, F SERIES)
277  0.001363          (SONATA, CAMRY)
278  0.001097         (SONATA, ACCORD)
279  0.001113  (RAM PICKUP, RAM WAGON)
280  0.001097       (TACOMA, FRONTIER)

[281 rows x 2 columns]


### evaluate the lift based on the mlxtend guide:
### http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/association_rules/

In [155]:
from mlxtend.frequent_patterns import association_rules

rules_ap = association_rules(frequent_itemsets_ap, metric="lift", min_threshold=1.1)
rules_fp = association_rules(frequent_itemsets_fp, metric="lift", min_threshold=1.1) # the min_threshold could be modified


In [156]:
rules_ap

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(MDX),(X5),0.013400,0.006770,0.001113,0.083041,12.265262,0.001022,1.083178
1,(X5),(MDX),0.006770,0.013400,0.001113,0.164352,12.265262,0.001022,1.180641
2,(7 SERIES),(TL),0.017051,0.016158,0.001128,0.066176,4.095560,0.000853,1.053563
3,(TL),(7 SERIES),0.016158,0.017051,0.001128,0.069835,4.095560,0.000853,1.056747
4,(CR-V),(TL),0.054477,0.016158,0.001034,0.018987,1.175097,0.000154,1.002884
...,...,...,...,...,...,...,...,...,...
111,"(PRIUS C, GT-R)",(CR-V),0.005141,0.054477,0.001348,0.262195,4.812970,0.001068,1.281536
112,"(CR-V, GT-R)",(PRIUS C),0.003009,0.067814,0.001348,0.447917,6.605089,0.001144,1.688488
113,(PRIUS C),"(CR-V, GT-R)",0.067814,0.003009,0.001348,0.019875,6.605089,0.001144,1.017208
114,(CR-V),"(PRIUS C, GT-R)",0.054477,0.005141,0.001348,0.024741,4.812970,0.001068,1.020098
